In [2]:
# Import libraries
'''
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
'''

# Pretty display for notebooks
'''
%matplotlib inline
'''

# Allows the use of display() for DataFrames
from IPython.display import display 

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
weather = pd.read_csv("../asset/weather.csv")
key = pd.read_csv("../asset/key.csv")
submission_example = pd.read_csv("../asset/sampleSubmission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("weather  data : ", weather.shape)
print("key  data : ", key.shape)
print("submission  data : ", submission_example.shape)

Train data :  (4617600, 4)
Test  data :  (526917, 3)
weather  data :  (20517, 20)
key  data :  (45, 2)
submission  data :  (526917, 2)


In [ ]:
# train.csv : date, store_nbr, item_nbr, units
# key.csv : store_nbr, station_nbr
# weather.csv : "station_nbr","date","tmax","tmin" ... 18개
# test.csv : date, store_nbr, item_nbr

In [6]:
train.tail(n=3)

,date,store_nbr,item_nbr,units
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [20]:
key[:2]

,store_nbr,station_nbr
0,1,1
1,2,14


In [7]:
weather.tail(n=3)

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
20514,18,2014-10-31,67,49,58,-4,40,50,7,0,0644,1738,,0.0,0.00,29.78,30.28,10.6,36,11.2
20515,19,2014-10-31,45,33,39,-6,24,32,26,0,0624,1646,RA SN,0.1,0.02,29.51,30.24,20.4,34,20.9
20516,20,2014-10-31,68,50,59,M,39,50,6,0,-,-,,0.0,0.00,29.57,30.27,10.5,36,11.2


In [28]:
data_tk = train.merge(key, left_on="store_nbr", right_on="store_nbr")
data_tk.tail(3)

,date,store_nbr,item_nbr,units,station_nbr
4617597,2014-10-31,35,109,0,5
4617598,2014-10-31,35,110,0,5
4617599,2014-10-31,35,111,0,5


In [50]:
data_tk.pivot_table(columns=["date","store_nbr","item_nbr"], value)

SyntaxError: positional argument follows keyword argument (<ipython-input-50-c989bdfb8364>, line 1)